5


In [13]:
import pandas as pd
from pathlib import Path
import requests
import time 

In [14]:
API_KEY = "df41212a-079d-4a5a-9c7e-5196b8ae2c24"

In [15]:
med_path = Path("/Users/theadiselab/Desktop/Jun Hyuk Lee_files/Files/medication_summary.xlsx")
cls_path = Path("/Users/theadiselab/Desktop/Jun Hyuk Lee_files/Files/unique_atc_code.xlsx")

med_df = pd.read_excel(med_path, sheet_name = "medication_summary")
cls_df = pd.read_excel(cls_path, sheet_name = "unique_atc_code")

cat_map = cls_df.set_index("broad_atc_code")["medication_category"].to_dict()


In [16]:
def fetch_detailed_atc_info(rxcui: int) -> list[str]:
    url = "https://rxnav.nlm.nih.gov/REST/rxclass/class/byRxcui.json"
    params = {"rxcui": rxcui, "relaSource": "ATC"}
    if API_KEY: 
        params["apiKey"] = API_KEY
    
    try: 
        resp = requests.get(url, params = params, timeout = 5)
        resp.raise_for_status()
        data = resp.json()
        infos = data.get("rxclassDrugInfoList", {}).get("rxclassDrugInfo", [])
        codes = []
        for info in infos: 
            cid = info.get("rxclassMinConceptItem", {}).get("classId", "")
            if cid:
                codes.append(cid)
        return codes
    except: 
        return []
    

In [17]:
med_df["atc_code"] = ""
med_df["broad_atc_code"] = ""
med_df["medication_category"] = ""

In [20]:
for idx, row in med_df.iterrows():
    mid = row["med_id"]
    try: 
        rxcui = int(mid)
    except: 
        continue

    codes = fetch_detailed_atc_info(rxcui)
    if not codes: 
        continue 

    atc = codes[0]
    broad = atc[:3]

    category = cat_map.get(broad, "")

    med_df.at[idx, "broad_atc_code"] = broad
    med_df.at[idx, "medication_category"] = category
    med_df.at[idx, "atc_code"] = atc

    time.sleep(0.1)

print(f"Done!")

Done!


In [21]:
out_path = Path("/Users/theadiselab/Desktop/Jun Hyuk Lee_files/Files/medication_summary_enriched.xlsx")
with pd.ExcelWriter(out_path, engine="xlsxwriter") as writer:
    med_df.to_excel(writer, index=False, sheet_name="medication_summary")

print(f"Done!")

Done!
